In [83]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from torch.utils.data import Dataset, DataLoader
import albumentations as A
%matplotlib inline
%config Completer.use_jedi = False

ROOT = "/home/lenin/code/hat_on_the_head/"
DATA = ROOT + "data/"
KP_DATA = DATA + "kaggle_keypoints/"

In [95]:
def load(path):
    img = cv2.imread(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


def valid_keypoints_plot(image, outputs, orig_keypoints, epoch):
    """
    This function plots the regressed (predicted) keypoints and the actual 
    keypoints after each validation epoch for one image in the batch.
    """
    # detach the image, keypoints, and output tensors from GPU to CPU
    image = image.detach().cpu()
    outputs = outputs.detach().cpu().numpy()
    orig_keypoints = orig_keypoints.detach().cpu().numpy()
    # just get a single datapoint from each batch
    img = image[0]
    output_keypoint = outputs[0]
    orig_keypoint = orig_keypoints[0]
    img = np.array(img, dtype='float32')
    img = np.transpose(img, (1, 2, 0))
    plt.imshow(img)
    
    output_keypoint = output_keypoint.reshape(-1, 2)
    orig_keypoint = orig_keypoint.reshape(-1, 2)
    for p in range(output_keypoint.shape[0]):
        plt.plot(output_keypoint[p, 0], output_keypoint[p, 1], 'r.')
        plt.plot(orig_keypoint[p, 0], orig_keypoint[p, 1], 'b.')
        
    plt.show()
    


def load_item(root, row):
    img = load(root + row.img)
    keypoints = [[float(e) for e in t.split(",")] for t in row.our_kpts.split(";")]
    return img, keypoints

def show(row):
    img, kpts = load_item(KP_DATA, row)
    show_w_kpts(img, kpts)

In [92]:
df = pd.read_csv(DATA + "our_train_kaggle_keypoints.csv")

if False:
    def join_kpts(row):
        kpts = []
        for kp in range(36, 92, 2):
            kpts.append(f"{sample[str(kp)]},{sample[str(kp + 1)]}")
        return ";".join(kpts)

    df["img"] = df["Unnamed: 0"]
    df["our_kpts"] = df.apply(join_kpts,axis=1)
    df = df.drop(columns=[str(i) for i in range(136)] + ["Unnamed: 0"])

    df.to_csv(DATA + "our_train_kaggle_keypoints.csv", index=False)

print(f"total images {len(df)}")    
df.head()

total images 3462


,img,our_kpts
0,Luis_Fonsi_21.jpg,"93.0,112.0;97.0,108.0;106.0,112.0;110.0,117.0;..."
1,Lincoln_Chafee_52.jpg,"93.0,112.0;97.0,108.0;106.0,112.0;110.0,117.0;..."
2,Valerie_Harper_30.jpg,"93.0,112.0;97.0,108.0;106.0,112.0;110.0,117.0;..."
3,Angelo_Reyes_22.jpg,"93.0,112.0;97.0,108.0;106.0,112.0;110.0,117.0;..."
4,Kristen_Breitweiser_11.jpg,"93.0,112.0;97.0,108.0;106.0,112.0;110.0,117.0;..."


In [103]:
def show_w_kpts(img, kpts):
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    keypoints = np.array(kpts)
    for j in range(len(keypoints)):
        plt.plot(keypoints[j, 0], keypoints[j, 1], 'b.')
    plt.show()

tfms = A.Compose([
    A.LongestMaxSize(448),
    A.ShiftScaleRotate(border_mode=0, value=0),
    A.RandomBrightnessContrast(p=0.2),
], keypoint_params=A.KeypointParams(format='xy'))


sample = df.sample(1).iloc[0]
orig_img, orig_kpts = load_item(KP_DATA + "training/", sample)

res = tfms(image=orig_img, keypoints=orig_kpts)
img_tfmd = res["image"]
kpts_tfmd = res["keypoints"]
show_w_kpts(img_tfmd, kpts_tfmd)

ValueError: Expected x for keypoint (155.0, 122.0, 0.0, 0) to be in the range [0.0, 154], got 155.0.

# Dataset

In [ ]:
class KeypointsDataset(Dataset):
    
    def __init__(self, root, df, aug=A.Compose([])):
        self.root = root
        self.df = df
        self.aug = aug

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = load(self.root + row['img'])
        kpts = []
        for kp in range(36, 92, 2):
            kpts.append([sample[str(kp)], sample[str(kp + 1)]])